<div style="text-align: center;"><br>
<img src="https://assets-global.website-files.com/62b9d45fb3f64842a96c9686/62d84db4aeb2f6552f3a2f78_Quantinuum%20Logo__horizontal%20blue.svg" width="200" height="200" /></div>

# Circuit Stitching

Circuit stitching is a workflow to "stitch" multiple jobs into 1 job using the Mid-Circuit Measurement & Reset (MCMR) feature. The first benenfit is to minimize H-System Quantum Credits (HQCs) consumption for experiments consisting of multiple jobs. The second benefit is the reduction of job runtime, because many jobs are stitched into one job submission. For each new job, the H-Series performs pre-checks and post-checks (calibrations to optimize the device for the job to be run). These checks add seconds to the job runtime. By combining many jobs into one job, only one pre-check and post-check is performed.

Stitching involves appending circuits depth-wise and width-wise (across the qubit register). There are two components to cost jobs in terms of HQCs:

1. a +5 contribution for each job submitted to H-Series;
2. a job-specific cost accounting for the number of gate operations and shots.

\begin{equation}
HQC = 5 + \frac{C}{5000} \left( N_{1q} + 10 N_{2q} + 5 N_m \right)
\end{equation}

<div style="text-align: center;">
    <img src="figures/circuit_stitching_figure_1.png" width="1600" />
</div>

The schematic above shows a basic stitching workflow. Instead of submitting multiple jobs to the H-Series device, the number of jobs can be minimized by stitching circuits together depth-wise. If the number of qubits needed is smaller than number of device qubits, then circuits can be stitched in parallel, across the qubit register. To ensure optimal performance, the following guidelines must be followed: 

* TKET programs of size ~4 MB;
* less then 10,000 2-qubit gate operations;
* a total cost of less then 30 HQCs;
* less than a specifc number of classical registers, defined on a per-device basis;
* contain classical registers with a width below a device-specific threshold.

**Contents:**
* [GHZ State Preparation](#GHZ-State-Preparation)
* [Backend Info](#Backend-Info)
* [Circuit Stitching](#Circuit-Stitching)
* [Job Costing and Submission](#Job-Costing-and-Submission)
* [Result Destitching](#Result-Destitching)
* [Conclusion](#Conclusion)

## GHZ State Preparation

The $N$-qubit GHZ state preparation, using the definition,

\begin{equation}
| {GHZ}_N \rangle = \frac{1}{\sqrt{2}} \left( {| 0 \rangle}^{\bigotimes N} + {| 1 \rangle}^{\bigotimes N} \right),
\end{equation}

where $N=5$.

In [180]:
from pytket.circuit import Circuit

n_qubits = 5
circ = Circuit(n_qubits)
circ.H(0)
for i in range(n_qubits-1):
    circ.CX(i,i+1)

The functions below generate the measurement circuits necessary to measure the GHZ state fidelity.

* `generate_population_circuit`: Generates a $N$-qubit circuit measuring the GHZ state in the computational basis.
* `generate_parity_circuits`: Generates $N$-sets of $N$-qubit circuits, each with all qubits measuring the GHZ state in the $X$-$Y$ plane.

In [299]:
from typing import Tuple, List
from pytket.circuit import Circuit

def generate_population_circuit(
    ghz_circuit: Circuit,
) -> Circuit:
    circuit = ghz_circuit.copy()
    circuit.add_barrier(circuit.qubits)
    circuit.measure_all()
    return circuit

def generate_parity_circuits(
    ghz_circuit: Circuit,
) -> Tuple[List[Circuit], List[float]]:
    circuits = []
    angles = []
    for k in range(1, ghz_circuit.n_qubits + 1):
        circuit = ghz_circuit.copy()
        angle = k / ghz_circuit.n_qubits
        for q in circuit.qubits:
            circuit.Rz(-angle, q)
            circuit.Ry(-0.5, q)
        circuit.measure_all()
        circuits += [circuit]
        angles += [angle]
    return circuits, angles

In [201]:
population_circuit = generate_population_circuit(circ)
parity_circuit_list, angles = generate_parity_circuits(circ)

## BackendInfo

An instance of the `QuantinuumBackend` is contructed and authenticated.

In [7]:
from pytket.extensions.quantinuum import QuantinuumBackend

backend = QuantinuumBackend(device_name="H1-1E")
backend.login()

This instance contains a `BackendInfo` attribute that reports useful device properties, such as maximum number of qubits, maximum number of classical bit registers and the maximum allowed width of any classical bit register. The `BackendInfo` instance can be exported to a dictionary and printed.

In [9]:
backend.backend_info.to_dict()

{'name': 'QuantinuumBackend',
 'device_name': 'H1-1E',
 'version': '0.33.0',
 'architecture': {'nodes': [['q', [0]],
   ['q', [1]],
   ['q', [2]],
   ['q', [3]],
   ['q', [4]],
   ['q', [5]],
   ['q', [6]],
   ['q', [7]],
   ['q', [8]],
   ['q', [9]],
   ['q', [10]],
   ['q', [11]],
   ['q', [12]],
   ['q', [13]],
   ['q', [14]],
   ['q', [15]],
   ['q', [16]],
   ['q', [17]],
   ['q', [18]],
   ['q', [19]]]},
 'gate_set': [66, 68, 36, 71, 8, 73, 76, 109, 14, 15, 16, 17, 18, 19, 20, 44],
 'n_cl_reg': 120,
 'supports_fast_feedforward': True,
 'supports_reset': True,
 'supports_midcircuit_measurement': True,
 'all_node_gate_errors': None,
 'all_edge_gate_errors': None,
 'all_readout_errors': None,
 'averaged_node_gate_errors': None,
 'averaged_edge_gate_errors': None,
 'averaged_readout_errors': None,
 'misc': {'wasm': True,
  'batching': True,
  'n_qubits_stb': 20,
  'max_classical_register_width': 32,
  'n_gate_zones': '5',
  'noise_specs': {'crosstalk_probability': {'p_crosstalk_meas'

The `get` method can be used directly on the `BackendInfo` instance to obtain the relevant quantities. The code cell below prints the number of qubits, number of classical registers, the max width of any classical register, in addition to device specific information.

In [19]:
backend_info =  backend.backend_info

device_qubits = backend_info.n_nodes
cl_reg_width = backend_info.get_misc("cl_reg_width")
n_cl_reg = backend_info.n_cl_reg
print(f"Number of device qubits:\t{device_qubits}")
print(f"Number of classical registers:\t{cl_reg_width}")
print(f"Max classical register width:\t{n_cl_reg}")
print(f"Supports MCMR:\t\t\t{backend_info.supports_midcircuit_measurement}")
print(f"Supports Fast Feed Forward:\t{backend_info.supports_fast_feedforward}")
print(f"Supports Reset:\t\t\t{backend_info.supports_reset}")

Number of device qubits:	20
Number of classical registers:	32
Max classical register width:	120
Supports MCMR:			True
Supports Fast Feed Forward:	True
Supports Reset:			True


## Circuit Stitching

The `circuit_stitching` method requires a list of circuits as input, in addition to a BackendInfo instance. The method will parallelise the circuits across the qubit register (stitch width-wise) and then stitch the circuits depth-wise. The `BackendInfo` instance informs the method on how many circuits can be parallelised. The method assumes that all circuits inputted as a list, have the same number of qubits.

In [223]:
from typing import List, Dict, Tuple

import numpy as np

from pytket.circuit import Circuit, CircBox, Qubit, BitRegister
from pytket.backends.backendinfo import BackendInfo


def reset_operations(
    n_qubits: int
) -> CircBox:
    r"""Generate a n-qubit CircBox instance containing OpType.Reset operations.

    :param n_qubits: Number of qubits used in the CircBox instance
    :param_type int:
    :returns: CircBox
    """
    circuit = Circuit(n_qubits)
    circuit.name = "Reset"
    for q in circuit.qubits:
        circuit.Reset(q)
    return CircBox(circuit)


def _add_circuit(
    circuit_instance: Circuit,
    circbox: CircBox,
    index: int,
    qubits: List[Qubit],
    resetbox: CircBox = None
):
    creg = circuit_instance.add_c_register(f"creg_{index}", len(qubits))
    circuit_instance.add_circbox(circbox, qubits + list(creg))
    if resetbox:
        circuit_instance.add_circbox(resetbox, qubits)


def circuit_stitching(
    input_circuits: List[Circuit], 
    backend_info: BackendInfo,
) -> Tuple[Circuit, Dict[int, BitRegister]]:
    r"""Generate a stitched circuit based on a list of input circuits. The circuit is 
    stitched width-wise and then depth-wise.

    :param input_circuit: Circuit instance to stitch `n` times.
    :param backend_info: BackendInfo instance containing information on number 
        of device qubits, number of allowed classical register and maximum width 
        for each classical register.
    :returns: Circuit
    """
    if all(input_circuits[0] == c for c in input_circuits[1:]):
        raise ValueError("All circuits should have the same number of qubits.")
    n_qubits_device = backend_info.architecture.nodes
    p = np.floor_divide(len(n_qubits_device), input_circuits[0].n_qubits)
    num_c_reg = 0
    circuit = Circuit(len(n_qubits_device))
    reset_box = reset_operations(input_circuits[0].n_qubits)

    circuits_list = [[] for _ in range(p)]
    for i, circ in enumerate(input_circuits):
        j = i % p
        circuits_list[j] += [circ]

    for k, circuits in enumerate(circuits_list):
        for i, c in enumerate(circuits):
            circ = c.copy()
            circ.name = f"box_{num_c_reg}"
            a = i * circ.n_qubits
            b = (i + 1) * circ.n_qubits
            qubits = circuit.qubits[a: b]
            if k == len(circuits_list) - 1:
                reset_box = None
            _add_circuit(circuit, CircBox(circ), num_c_reg, qubits, resetbox=reset_box)
            num_c_reg += 1
    return circuit

The `circuit_stitching method` is used to stitch the 5-qubit GHZ population circuit and 5 GHZ parity circuits.

In [224]:
input_circuits = [population_circuit] + parity_circuit_list
stitched_circuit = circuit_stitching(input_circuits, backend.backend_info, parallelize_across_register=True)

The resulting circuit contains 6 distinct classical registers. Each classical register corresponds to a subcircuit in the stitched circuit.

In [218]:
registers = stitched_circuit.c_registers
registers.sort(key=lambda a: int(a.name.split("_")[-1]))

In [300]:
print(registers)

[BitRegister("creg_0", 5), BitRegister("creg_1", 5), BitRegister("creg_2", 5), BitRegister("creg_3", 5), BitRegister("creg_4", 5), BitRegister("creg_5", 5)]


The classical registers are required to post-process the result after H-Series execution is complete. The classical register information should be saved to `JSON`, enabling this information to be used across python sessions. The ordering of the classical registers needs to preserved in JSON format, and the a record should be kept of the name and size of each classical register.

In [225]:
import json

register_data = {reg.name: reg.size for reg in registers}

with open("stitched_registers.json", "w") as json_io:
    json.dump(register_data, json_io)

## Costing Analysis

The stitched circuit is compiled to use the native H-Series gateset. The individual circuits, used to construct the stitched circuit, are also compiled for the costing analysis. 

In [303]:
compiled_stitched_circuit = backend.get_compiled_circuit(stitched_circuit, optimisation_level=2)
compiled_circuits = backend.get_compiled_circuits([population_circuit] + parity_circuit_list, optimisation_level=2)

The all circuits will be submitted and costed with 100 shots each.

In [306]:
n_shots = 100

The function below calculates the HQC cost of a circuit, based on the number of operations and number of shots. As mentioned previously, there is a +5 overhead for each new job submitted to H-Series for execution.

In [304]:
from pytket.circuit import Circuit, OpType
from pytket.backends.backendinfo import BackendInfo


def _cost(
    circuit: Circuit,
    n_shots: int
) -> int:
    r"""Local estimation of HQCs (H-System Quantum Credits) 
    needed to execute stitched circuit.

    :param circuit: circuit instance.
    :returns: int
    """
    n1q = circuit.n_1qb_gates()
    n2q = circuit.n_2qb_gates()
    nm = circuit.n_gates_of_type(OpType.Measure)
    return n_shots/5000 * (n1q + 10 * n2q + 5 * nm) + 5

Each measurement circuit approximately requires 40 HQCs, but the stitched circuit requires only 15 HQCs. By stitching the measurement circuits, an 2.5x decrease is observed in the required number of HQCs. 

In [312]:
sum(_cost(c, n_shots) for c in compiled_circuits)

39.32000000000001

In [313]:
_cost(compiled_stitched_circuit, n_shots)

14.32

The function below verifies the stitched circuit satisfies the various performance constraints before submission to H-Series:

* less then 10,000 2-qubit gate operations;
* a total cost of less then 30 HQCs;
* less than a specifc number of classical registers, defined on a per-device basis;
* contain classical registers with a width below a device-specific threshold.

In [ ]:
def verify_stitched_circuit(
    circuit: Circuit,
    n_shots: int,
    backend_info: BackendInfo
) -> Tuple[bool, Dict[str, float | int]]:
    """Verify the stitched circuit satisfies basic 
    constraints before submission to H-Series.

    :param circuit: `Circuit` instance to stitch `n` times.
    :param backend_info: `BackendInfo` instance containing information on number 
        of device qubits, number of allowed classical register and maximum width 
        for each classical register.
    :returns: bool
    """
    check = True
    data = {}
    data["cost"] = [_cost(circuit, n_shots), 30]
    data["n_2qb_gates"] = [circuit.n_2qb_gates(), 10000]
    data["c_registers"] = [len(circuit.c_registers), backend_info.n_cl_reg]
    data["max_cl_reg_width"] = [max([a.size for a in circuit.c_registers]), backend_info.get_misc("cl_reg_width")]

    if _cost(circuit, n_shots) > 30:
        check = False
    if circuit.n_2qb_gates() > 10000:
        check = False
    if len(circuit.c_registers) > backend_info.n_cl_reg:
        check = False
    if all(a.size > backend_info.get_misc("cl_reg_width") for a in circuit.c_registers):
        check = False
        
    return check, data

The method returns a boolean and a dictionary. The boolean determines whether the stitched circuit satisfies the performance constraints before submission (True is success). The dictionary reports the cost, two-qubit gate count, number of classical registers and maximum size across all classical registers. In addition, the limit for each property is reported.

In [314]:
check, data = verify_stitched_circuit(compiled_stitched_circuit, 100, backend.backend_info)
print(check)
print(data)

True
{'cost': [14.32, 30], 'n_2qb_gates': [24, 10000], 'c_registers': [6, 120], 'max_cl_reg_width': [5, 32]}


`QuantinuumBackend` can be used to cost the stitched circuit as well. This will be similar to the estimate above.

In [276]:
cost_hqc = backend.cost(compiled_stitched_circuit, n_shots=100, syntax_checker="H1-1SC")
print(cost_hqc)

17.78


`QuantinuumBackend` is used to submit the circuit for execution to H-Series.

In [290]:
handle = backend.process_circuit(compiled_stitched_circuit, n_shots=100)

## Result Destitching

`QuantinuumBackend` is used to check the status of the stitched circuit job.

In [292]:
backend.circuit_status(handle)

CircuitStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='{"name": "job", "submit-date": "2024-08-22T17:18:28.109559", "result-date": "2024-08-22T17:18:45.585046", "queue-position": null, "cost": "17.78", "error": null}', error_detail=None, completed_time=None, queued_time=None, submitted_time=None, running_time=None, cancelled_time=None, error_time=None, queue_position=None)

`QuantinuumBackend` can be used to retrieve the result from the backend.

In [294]:
result = backend.get_result(handle)

The register JSON is opended and loaded into this jupyter session.

In [295]:
import json

with open("stitched_registers.json", "r") as json_io:
    register_data = json.load(json_io)

The JSON data is used to instiate a list of `BitRegister` instances.

In [296]:
from pytket.circuit import BitRegister

register_list = [BitRegister(name, size) for name, size in register_data.items()]

The result for the stitched circuit needs to be *destitched* into effective measurement results for each individual circuit.

In [297]:
from pytket.backends.backendresult import BackendResult
from pytket.utils.outcomearray import OutcomeArray

destitched_results = []
for reg in register_list:
    outcome_array = OutcomeArray.from_readouts(result.get_shots(reg))
    destitched_results += [BackendResult(shots=outcome_array)]

The dictionaries below report the effective measurement result for each individual measurement circuit. The effective measurement result is matches the ordering of the measurement circuits as inputted into the `circuit_stitching` function.

In [298]:
for r in destitched_results:
    print(r.get_distribution())

{(0, 0, 0, 0, 0): 0.47, (1, 1, 1, 1, 1): 0.53}
{(0, 0, 0, 0, 0): 0.1, (0, 0, 0, 0, 1): 0.01, (0, 0, 0, 1, 1): 0.05, (0, 0, 1, 0, 1): 0.07, (0, 0, 1, 1, 0): 0.07, (0, 1, 0, 0, 1): 0.05, (0, 1, 0, 1, 0): 0.07, (0, 1, 1, 0, 0): 0.04, (0, 1, 1, 1, 1): 0.06, (1, 0, 0, 0, 1): 0.06, (1, 0, 0, 1, 0): 0.04, (1, 0, 1, 0, 0): 0.05, (1, 0, 1, 1, 1): 0.04, (1, 1, 0, 0, 0): 0.07, (1, 1, 0, 1, 1): 0.06, (1, 1, 1, 0, 1): 0.06, (1, 1, 1, 1, 0): 0.1}
{(0, 0, 0, 0, 1): 0.09, (0, 0, 0, 1, 0): 0.04, (0, 0, 1, 0, 0): 0.02, (0, 0, 1, 1, 1): 0.05, (0, 1, 0, 0, 0): 0.1, (0, 1, 0, 1, 0): 0.01, (0, 1, 0, 1, 1): 0.08, (0, 1, 1, 0, 1): 0.07, (0, 1, 1, 1, 0): 0.08, (1, 0, 0, 0, 0): 0.05, (1, 0, 0, 1, 1): 0.06, (1, 0, 1, 0, 0): 0.01, (1, 0, 1, 0, 1): 0.03, (1, 0, 1, 1, 0): 0.03, (1, 1, 0, 0, 1): 0.03, (1, 1, 0, 1, 0): 0.05, (1, 1, 1, 0, 0): 0.06, (1, 1, 1, 1, 0): 0.01, (1, 1, 1, 1, 1): 0.13}
{(0, 0, 0, 0, 0): 0.01, (0, 0, 0, 0, 1): 0.04, (0, 0, 0, 1, 0): 0.1, (0, 0, 1, 0, 0): 0.09, (0, 0, 1, 1, 1): 0.06, (0, 1, 0, 0

<!-- ## Classical Shadows -->

In [27]:
# from typing import Tuple, List

# from numpy.random import choice

# from pytket.circuit import Circuit, Bit, Qubit
# from pytket.partition import MeasurementSetup, MeasurementBitMap
# from pytket.pauli import Pauli, QubitPauliString
# from pytket.utils.operators import QubitPauliOperator


# def create_measurement_basis(
#     qps: QubitPauliString, 
#     n_qubits: int
# ) -> Circuit:
#     circuit = Circuit(n_qubits)
#     for qubit, pauli in qps.map.items():
#         bit = Bit(qubit.index[0])
#         circuit.add_bit(bit)
#         if pauli == Pauli.X:
#             circuit.H(qubit)
#         elif pauli == Pauli.Y:
#             circuit.V(qubit)
#         circuit.Measure(qubit, bit)
#     return circuit


# def construct_randomized_shadows(
#     measurement_budget: int,
#     n_qubits: int,
# ) -> Tuple[List[Circuit], MeasurementSetup]:
#     measurement_setup = MeasurementSetup()
#     circuit_list = []
#     PAULIS = [Pauli.X, Pauli.Y, Pauli.Z]
#     for j in range(measurement_budget):
#         qps = QubitPauliString({Qubit(i): choice(PAULIS) for i in range(n_qubits)})
#         measurement_circ = create_measurement_basis(qps, n_qubits)
#         circuit_list += [measurement_circ]
#         measurement_setup.add_measurement_circuit(measurement_circ)
#         if qps == QubitPauliString():
#             continue
#         bits = []
#         qps_items = qps.map.items()
#         if len(qps_items) != len(qps.map):
#             continue
#         for qubit, _ in qps_items:
#             bits += [qubit.index[0]]
#         if len(bits) > 0:
#             bits.sort()
#             bitmap = MeasurementBitMap(j, bits)
#             measurement_setup.add_result_for_term(qps, bitmap)
#     return circuit_list, measurement_setup

In [28]:
# from pytket.circuit import Circuit

# n_qubits = 5
# circ = Circuit(n_qubits)
# circ.H(0)
# for i in range(n_qubits-1):
#     circ.CX(i,i+1)

In [29]:
# measurement_budget = 1000
# mcircs, ms = construct_randomized_shadows(measurement_budget, n_qubits)

In [30]:
# handles = []
# for i, mc in enumerate(mcircs):
#     c = circ.copy()
#     c.name = f"job-{i}"
#     c.append(mc)
#     cc = backend.get_compiled_circuit(c, optimisation_level=2)
#     h = backend.process_circuit(cc, n_shots=1)
#     handles += [h]

In [31]:
# import json

# with open("json_handles_2.json", "w") as json_io:
#     json.dump([str(h) for h in handles], json_io)

In [ ]:
# results = backend.get_results(handles)

In [45]:
# from pytket.utils import OutcomeArray
# from pytket.backends.backendresult import BackendResult

# shots_array = []
# for r in results:
#     shots_array.extend(r.get_shots().tolist())

# result = BackendResult(shots=OutcomeArray.from_readouts(shots_array))

In [ ]:
# data = {}
# for v in ms.results.values():
#     print(v)

In [51]:
# @hiddencell

# import numpy as np
# import sympy as sp

# from pytket.backends.backendresult import BackendResult
# from pytket.partition import MeasurementSetup

# def snapshot_state(
#     results: BackendResult, 
#     measurement_setup: MeasurementSetup
# ):
#     """
#     Helper function for `shadow_state_reconstruction` that reconstructs
#      a state from a single snapshot in a shadow.

#     Implements Eq. (S44) from https://arxiv.org/pdf/2002.08953.pdf

#     Args:
#         b_list (array): The list of classical outcomes for the snapshot.
#         obs_list (array): Indices for the applied Pauli measurement.

#     Returns:
#         Numpy array with the reconstructed snapshot.
#     """
#     num_qubits = len(b_list)

#     # computational basis states
#     zero_state = np.array([[1, 0], [0, 0]])
#     one_state = np.array([[0, 0], [0, 1]])

#     # local qubit unitaries
#     phase_z = np.array([[1, 0], [0, -1j]], dtype=complex)
#     hadamard = np.array([[1, 1], [1, -1]])
#     identity = np.array([[1, 0], [0, 1]])

#     # undo the rotations that were added implicitly to the circuit for the Pauli measurements
#     unitaries = [hadamard, hadamard @ phase_z, identity]

#     # reconstructing the snapshot state from local Pauli measurements
#     rho_snapshot = [1]
#     for i in range(num_qubits):
#         r = results[].get_shots()
#         state = zero_state if b_list[i] == 1 else one_state
#         U = unitaries[int(obs_list[i])]

#         # applying Eq. (S44)
#         local_rho = 3 * (U.conj().T @ state @ U) - identity
#         rho_snapshot = np.kron(rho_snapshot, local_rho)

#     return rho_snapshot


# def shadow_state_reconstruction(shadow):
#     """
#     Reconstruct a state approximation as an average over all snapshots in the shadow.

#     Args:
#     :param shadow: (tuple): A shadow tuple obtained from `calculate_classical_shadow`.

#     Returns:
#         Numpy array with the reconstructed quantum state.
#     """
#     num_snapshots, num_qubits = shadow[0].shape

#     # classical values
#     b_lists, obs_lists = shadow

#     # Averaging over snapshot states.
#     shadow_rho = np.zeros((2 ** num_qubits, 2 ** num_qubits), dtype=complex)
#     for i in range(num_snapshots):
#         shadow_rho += snapshot_state(b_lists[i], obs_lists[i])

#     return shadow_rho / num_snapshots

<div align="center"> &copy; 2024 by Quantinuum. All Rights Reserved. </div>